In [ ]:
import xarray as xr
import os
from pathlib import Path
from tqdm import tqdm

# ── 0. Set up your paths ────────────────────────────────────────────────
BASE      = Path("to_folder")
IN_DIR    = BASE / "folder"
OUT_DIR = BASE / "HI_India_daymean"     # where to write the one-file‐per‐year outputs
OUT_DIR.mkdir(exist_ok=True)

# ── 1. Loop over each year’s hourly file ─────────────────────────────────
for year in range(2003, 2021):
    infile = IN_DIR / f"HI_hourly_{year}_MAM_IST.nc"
    if not infile.exists():
        print(f"⚠️  Missing input for {year}, skipping.")
        continue

    # 2. Open the hourly HI dataset (lazy)
    ds = xr.open_dataset(infile)
    hi = ds["HI_noaa"]  # dims: time, y, x

    # 3. Restrict to the hours 13–16 (local/UTC whichever your data are in)
    hi_daytime = hi.sel(
        time=hi.time.dt.hour.isin([12,13,14,15,16])
    )

    # 4. Group by calendar day, compute daily mean of those hours
    hi_daily = hi_daytime.groupby("time.dayofyear").mean("time")

    # 5. Mean across all days in MAM → one 2-D field
    hi_daymean = hi_daily.mean("dayofyear", keep_attrs=True)
    hi_daymean.name = "HI_daymean_MAM"

    # 6. Write out to a 2-D NetCDF
    outfile = OUT_DIR / f"HI_daymean_MAM_{year}_IST.nc"
    hi_daymean.to_netcdf(
        outfile,
        engine="netcdf4",
        format="NETCDF4",
        encoding={"HI_daymean_MAM": {"zlib": True, "complevel": 4}}
    )
    print(f"✓ {year}: wrote → {outfile.name}")

    
    
#!/usr/bin/env python3
import xarray as xr
import rioxarray       # registers the .rio accessor
import geopandas as gpd
from pathlib import Path

# ── 0. Paths ───────────────────────────────────────────────────────────────

NC_DIR    = BASE / "folder_path"       # your yearly .nc inputs
SHAPEFILE = Path(
    "folder/india_simple_outline.shp"
)
OUT_DIR   = BASE / "HI_NOAA_MAX_12_16" / "HI"  # where to write the .tifs
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ── 1. Read India boundary once ────────────────────────────────────────────
gdf = gpd.read_file(SHAPEFILE).to_crs("EPSG:4326")

# ── 2. Loop over years ─────────────────────────────────────────────────────
for year in range(2003, 2021):
    nc_file = NC_DIR / f"HI_daymean_MAM_{year}_IST.nc"
    if not nc_file.exists():
        print(f"⚠️  Missing NetCDF for {year}, skipping")
        continue

    print(f"▶ Processing {year}…")

    # 2a) Open the NetCDF and grab the DataArray
    da = xr.open_dataset(nc_file)["HI_daymean_MAM"]

    # 2b) Give it a CRS and clip to the India polygon
    da_clipped = (
        da
        .rio.write_crs("EPSG:4326")
        .rio.clip(gdf.geometry, gdf.crs, drop=True)
    )

    # 2c) Build the output path
    out_tif = OUT_DIR / f"HI_daymean_{year}_MAM_12-16.tif"

    # 2d) Export as a single‐band LZW‐compressed GeoTIFF
    da_clipped.rio.to_raster(
        out_tif,
        driver="GTiff",
        compress="LZW",
        dtype="float32",
        nodata=-9999
    )

    print(f"   ✓ saved {out_tif.name}")

print("\nAll done — your GeoTIFFs are in:", OUT_DIR)


import rasterio
from rasterio.merge import merge

# 1. Input & output directories
src_dir = "folder_LC_outputs"
dst_dir = "folder/LC_mosaics"
os.makedirs(dst_dir, exist_ok=True)

# 2. Regex to extract year & part
pattern = re.compile(r"landcover_(\d{4})_part([1-4])-.*\.tif$")


# 3. Collect and group files
groups = {}
for fp in glob.glob(os.path.join(src_dir, "*.tif")):
    fn = os.path.basename(fp)
    m = pattern.match(fn)
    if not m:
        continue
    year, part = m.groups()
    groups.setdefault((year, part), []).append(fp)

# 4. Mosaic each group
for (year, part), tif_list in groups.items():
    print(f"Mosaicking Year={year} Part={part} ({len(tif_list)} tiles)…")
    src_files = [rasterio.open(p) for p in tif_list]
    mosaic, out_trans = merge(src_files)
    
    # 5. Update metadata
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width":  mosaic.shape[2],
        "transform": out_trans
    })
    
    # 6. Write mosaic
    out_path = os.path.join(
        dst_dir,
        f"landcover_{year}_part{part}_mosaic.tif"
    )
    with rasterio.open(out_path, "w", **out_meta) as dst:
        dst.write(mosaic)
    
    # 7. Close sources
    for src in src_files:
        src.close()

print("All done! Check", dst_dir)

#!/usr/bin/env python3
"""
Year-by-year ensemble downscaling of Heat Index:
- RandomForest for central HI (mean loss)
- GradientBoostingRegressor (quantile loss) for high-HI tail

Predictors: LST + WSA + POP + GEO + RAD + DIST2COAST + DPTnorm + IMP

For each year (2003–2020):
  1. Sample SAMPLE_FRAC of all valid India pixels
  2. Stack predictors & target
  3. Train/test split
  4. Oversample high-HI tail for quantile model
  5. Fit RF (mean) and GBR (quantile)
  6. Combine predictions: use GBR where it exceeds RF, otherwise RF
  7. Predict and write full-grid map
"""

import os
import gc
import random
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.features import geometry_mask
from shapely.geometry import MultiPolygon, Polygon
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

# ──────────────────────────────────────────────────────────────────────────
# 0. Paths & settings
# ──────────────────────────────────────────────────────────────────────────

RF_OUTDIR       = "folder/Ensemble_peryear"
INDIA_SHP       = os.path.join(ROOT, "india_simple_outline/india_simple_outline.shp")
YEARS           = list(range(2003, 2021))
SAMPLE_FRAC     = 1.0       # sample fraction
RF_N_EST        = 150       # RF trees
GBR_N_EST       = 300       # GBR trees
GBR_LR          = 0.05      # learning rate for quantile
GBR_MAX_DEPTH   = 5
RND             = 7
QUANTILE_ALPHA  = 0.9       # focus on 90th percentile
EXTREME_FRAC    = 0.3       # fraction of training for extremes

random.seed(RND)
np.random.seed(RND)
os.makedirs(RF_OUTDIR, exist_ok=True)

# ──────────────────────────────────────────────────────────────────────────
# 1. Build 1 km template & mask
# ──────────────────────────────────────────────────────────────────────────
india_gdf = gpd.read_file(INDIA_SHP).to_crs("EPSG:4326")
poly = india_gdf.geometry.unary_union
if isinstance(poly, Polygon):
    poly = MultiPolygon([poly])

def open_1km_template():
    sample = os.path.join(ROOT, "LST/LST_INDIA_2007_MAM.tif")
    with rasterio.open(sample) as src:
        prof = src.profile
        h, w = src.height, src.width
        ys = np.linspace(src.bounds.top, src.bounds.bottom, h)
        xs = np.linspace(src.bounds.left, src.bounds.right, w)
    return (h, w), prof, ys, xs

(grid_h, grid_w), grid_profile, y_coords, x_coords = open_1km_template()

india_mask = geometry_mask([
    poly
], invert=True,
    transform=grid_profile['transform'],
    out_shape=(grid_h, grid_w)
)
lat_grid = np.repeat(y_coords.reshape(-1,1), grid_w, axis=1)
lon_grid = np.repeat(x_coords.reshape(1,-1), grid_h, axis=0)

# ──────────────────────────────────────────────────────────────────────────
# 2. Raster reader & static predictors
# ──────────────────────────────────────────────────────────────────────────

def read_raster(path, match_profile=None):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(np.float32)
        if match_profile:
            aligned = (
                src.transform == match_profile['transform'] and
                src.crs      == match_profile['crs'] and
                src.width    == match_profile['width'] and
                src.height   == match_profile['height']
            )
            if not aligned:
                dst = np.full((match_profile['height'], match_profile['width']),
                              np.nan, np.float32)
                reproject(
                    source=arr,
                    destination=dst,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=match_profile['transform'],
                    dst_crs=match_profile['crs'],
                    resampling=Resampling.bilinear
                )
                return dst
        return arr

# static layers
geo  = read_raster(os.path.join(ROOT, "GEO_INDIA_1km.tif"), grid_profile)
dist = read_raster(os.path.join(ROOT, "distance_tocoast/India_Coast_Distance_km.tif"), grid_profile)
geo[~india_mask]  = np.nan
dist[~india_mask] = np.nan

# ──────────────────────────────────────────────────────────────────────────
# 3. Loop over years
# ──────────────────────────────────────────────────────────────────────────
for yr in tqdm(YEARS, desc="Processing years"):
    print(f"\n--- Year {yr} ---")
    # load annual rasters
    lst     = read_raster(f"{ROOT}/LST/LST_INDIA_{yr}_MAM.tif", grid_profile)
    wsa     = read_raster(f"{ROOT}/WSA/WSA_INDIA_{yr}_MAM.tif", grid_profile)
    pop     = read_raster(f"{ROOT}/POP_outputs/POP_INDIA_{yr}.tif", grid_profile)
    rad     = read_raster(f"{ROOT}/ERA_RAD_outputs_norm/RAD_INDIA_{yr}_MAM.tif", grid_profile)
    dptnorm = read_raster(f"{ROOT}/ERA_DPTnorm_outputs/DPTNORM_INDIA_{yr}_MAM.tif", grid_profile)
    imp     = read_raster(f"{ROOT}/LC_final/landcover_{yr}.tif", grid_profile)
    hi = read_raster(  f"/home/udit/Desktop/Ongoing_Projects/Ashish/Heat_Index_Work/HI_NOAA_MAX_12_16/HI/HI_daymean_{yr}_MAM_12-16.tif", grid_profile)

    # valid mask & sample
    valid = (
        india_mask & np.isfinite(lst) & np.isfinite(wsa)
        & np.isfinite(pop) & np.isfinite(geo)
        & np.isfinite(rad) & np.isfinite(dist)
        & np.isfinite(dptnorm) & np.isfinite(imp)
        & np.isfinite(hi)
    )
    idx = np.where(valid.ravel())[0]
    if idx.size == 0:
        print(" No valid pixels; skipping.")
        continue
    nsel = int(SAMPLE_FRAC * idx.size)
    print(f" Sampled {nsel} valid pixels")
    sel  = np.random.choice(idx, nsel, replace=False)

    # stack features & target
    X = np.column_stack([
        np.full(nsel, yr, dtype=np.int16),
        lat_grid.ravel()[sel], lon_grid.ravel()[sel],
        lst.ravel()[sel], wsa.ravel()[sel],
        pop.ravel()[sel], geo.ravel()[sel],
        rad.ravel()[sel], dist.ravel()[sel],
        dptnorm.ravel()[sel], imp.ravel()[sel]
    ]).astype(np.float32)
    y = hi.ravel()[sel].astype(np.float32)

    # train/test split
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.3, random_state=RND
    )
    print(f" Train/test split: {X_tr.shape[0]} train, {X_te.shape[0]} test")

    # RF for mean HI
    rf = RandomForestRegressor(
        n_estimators=RF_N_EST,
        max_features='sqrt',
        max_depth=None,
        random_state=RND,
        n_jobs=-1
    )
    rf.fit(X_tr, y_tr)
    rf_pred_te = rf.predict(X_te)
    rf_mae = mean_absolute_error(y_te, rf_pred_te)
    print(f" RF hold-out MAE: {rf_mae:.3f} °C")

    # oversample extremes & quantile GBR
    thr = np.quantile(y_tr, QUANTILE_ALPHA)
    ext_idx    = np.where(y_tr > thr)[0]
    nonext_idx = np.where(y_tr <= thr)[0]
    n_ext      = int(EXTREME_FRAC * len(y_tr))
    n_non      = len(y_tr) - n_ext
    sel_ext    = np.random.choice(ext_idx,    n_ext,   replace=(n_ext>len(ext_idx)))
    sel_non    = np.random.choice(nonext_idx, n_non, replace=False)
    print(f" Extreme threshold ({QUANTILE_ALPHA*100:.0f}th percentile): {thr:.2f} °C")
    print(f" Oversampling: {len(ext_idx)} available extremes -> {n_ext} used, {n_non} non-extreme")
    idx_bal    = np.concatenate([sel_ext, sel_non])
    Xb, yb     = X_tr[idx_bal], y_tr[idx_bal]

    gbr = GradientBoostingRegressor(
        loss='quantile',
        alpha=QUANTILE_ALPHA,
        learning_rate=GBR_LR,
        max_depth=GBR_MAX_DEPTH,
        n_estimators=1000,            # an upper bound
        n_iter_no_change=10,          # stop if no improvement
        validation_fraction=0.1,      # hold out 10% of your Xb,yb
        random_state=RND
    )

    gbr.fit(Xb, yb)
    gbr_pred_te = gbr.predict(X_te)
    gbr_mae = mean_absolute_error(y_te, gbr_pred_te)
    print(f" GBR (quantile) hold-out MAE: {gbr_mae:.3f} °C")

    # ensemble combine
    pred_te = np.where(gbr_pred_te > rf_pred_te, gbr_pred_te, rf_pred_te)
    ens_mae = mean_absolute_error(y_te, pred_te)
    print(f" Ensemble hold-out MAE: {ens_mae:.3f} °C")

    # full-grid predict
    Npix = grid_h * grid_w
    Xg = np.column_stack([
        np.full(Npix, yr, dtype=np.int16),
        lat_grid.ravel(), lon_grid.ravel(),
        lst.ravel(), wsa.ravel(), pop.ravel(), geo.ravel(),
        rad.ravel(), dist.ravel(), dptnorm.ravel(), imp.ravel()
    ]).astype(np.float32)

    # flatten valid mask and prepare output container
    valid_flat = valid.ravel()
    pred_flat = np.full(Npix, np.nan, dtype=np.float32)

    # only predict on valid pixels
    Xg_valid = Xg[valid_flat]
    rf_valid = rf.predict(Xg_valid)
    gbr_valid = gbr.predict(Xg_valid)
    ens_valid = np.where(gbr_valid > rf_valid, gbr_valid, rf_valid)

    # scatter predictions back into full grid
    pred_flat[valid_flat] = ens_valid
    pred = pred_flat.reshape(grid_h, grid_w)

    # write output TIFF
    out_fp = os.path.join(RF_OUTDIR, f"HI_ensemble_{yr}.tif")
    prof   = grid_profile.copy()
    prof.update(dtype='float32', count=1, compress='LZW', nodata=np.nan)
    with rasterio.open(out_fp, 'w', **prof) as dst:
        dst.write(pred, 1)

    # cleanup
    del lst, wsa, pop, rad, dptnorm, imp, hi, X, y, X_tr, X_te, y_tr, y_te, rf, gbr, Xg, pred
    gc.collect()

print("\n🎉 All years done with ensemble!")
